# SMAI Assignment - 2

## Question 2: Gaussian Mixture Models

Resources: 
- https://youtu.be/qMTuMa86NzU
- https://youtu.be/ZBLyXgjBx3Q

Reference: https://scikit-learn.org/stable/modules/mixture.html 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
import cv2
from tqdm import tqdm
import time


### Part 1: Gaussian Mixture Models

We'll attempt to solve the task of background subtraction using Gaussian Mixture Models. Before that, you will need to implement the Gaussian Mixture Model algorithm from scratch.

Some details: 
- Try to implement GMMs using Multi-variate Gaussian Distributions, the following tasks in the assignment are possible to implement using the Univariate version too but it might be bit inaccurate as explained below.
    - To clarify further, we could treat each pixel in our RGB image as our data point with [R, G, B] channels as the dimensions to the Multi-variate data point, and we would easily get predictions for each pixel location using Multi-variate approach.
    - Or, we could treat every single value in the given RGB image as a data point independent of what channel the belong to and consider them as Uni-variate data point, and get prediction using the Uni-variate approach.
    But this affects our prediction, since we can't simply make per pixel predtions anymore, because for every pixel location we would now have 3 different predictions.
    - To get around this, you could convert your image to Grayscale and then we would only have one channel/value corresponding to each pixel location, which would now allow us to use the Uni-variate approach for prediction, but this also means loss in information which would affect our quality of predictions.
    - Try to have a class based implementation of GMM, this would really help you in Background Subtraction task. You can get some general ideas on how to structure your class by looking at `sklearn.mixture.GaussianMixture` documentation and source code.
- The following code cell has a rough template to get you started with the implementation. You are free to change the structure of the code, this is just a suggestion to help you get started.


TLDR: You may implement the univariate version of GMMs, but it might not be as accurate as the multivariate version and it is recommended to try and implement the multivariate version.

In [2]:
class GMM(object):
    
    weights = None
    means = None
    covars = None
    
    def __init__(self, n_components=1, tol=1e-3, max_iter=100):
        """
        n_components: The number of mixture components.
        tol: The convergence threshold
        """
        self.n_components = n_components
        self.tol = tol
        self.max_iter = max_iter
        self.reg_covar = 1e-6

    def initialize_params(self, X):
        """
        X : A collection of `N` training data points, each with dimension `d`.
        """
        n_samples, self.d = X.shape
        # rand_indices = np.random.choice(X.shape[0], size=self.n_components, replace=False)
        # self.means = X[rand_indices]
        kmeans = KMeans(n_clusters=self.n_components,n_init='auto',init='k-means++')
        kmeans.fit(X)
        self.means = kmeans.cluster_centers_
        variances = np.var(X, axis=0)
        self.covars = np.zeros((self.n_components, self.d, self.d))
        for k in range(self.n_components):
            np.fill_diagonal(self.covars[k], variances)
        self.weights = np.ones(self.n_components) / self.n_components
        
    
    def E_step(self, X):
        """
        Find the Expectation of the log-likelihood evaluated using the current estimate for the parameters.
        """
        posteriors = np.zeros((X.shape[0], self.n_components))
        for i in range(self.n_components):
            posteriors[:,i] = np.log(self.weights[i])+self.log_multivariate_normal(X, self.means[i], self.covars[i])
        # print(posteriors)
        log_likelihoods = np.log(np.sum(np.exp(posteriors), axis=1))
        posteriors -= log_likelihoods[:, np.newaxis]
        self.posteriors = posteriors

    def log_multivariate_normal(self, X, mean, covariance):
   
        n_samples = X.shape[1]
        log_det = np.log(np.linalg.det(covariance)+ 1e-6)
        log_norm = -0.5 * (n_samples * np.log(2 * np.pi) + log_det)
        deviation = X - mean
        log_exponent = -0.5 * np.sum(deviation @ np.linalg.inv(covariance) * deviation, axis=1)
        # print(log_norm + log_exponent)
        return log_norm + log_exponent
        
    def M_step(self, X):
        """
        Updates parameters maximizing the expected log-likelihood found on the E step.
        """
        self.posteriors = np.exp(self.posteriors)
        sum_of_posterios = self.posteriors.sum(axis=0)
        self.means = np.dot(self.posteriors.T, X) / sum_of_posterios[:, np.newaxis]
        for i in range(self.n_components):
            deviation = X - self.means[i]
            self.covars[i] = np.dot(self.posteriors[:, i] * deviation.T, deviation) / sum_of_posterios[i]
            self.covars[i].flat[:: self.d + 1] += self.reg_covar
        self.weights = sum_of_posterios / sum_of_posterios.sum()

    def fit(self, X, y=None):
        """
        Fit the parameters of the GMM on some training data.
        """
        self.initialize_params(X)
        prev_log_likelihood = None
        self.iterations = 0
        for _ in range(self.max_iter):
            self.iterations +=1
            self.E_step(X)
            self.M_step(X)
            self.E_step(X)
            log_likelihood = np.sum(self.posteriors)
            if prev_log_likelihood is not None and np.abs(log_likelihood - prev_log_likelihood) < self.tol:
                break
            prev_log_likelihood = log_likelihood
        
    def predict(self, X):
        """
        Predict the labels for the data samples in X using trained model.
        """
        self.E_step(X)
        preds = np.argmax(self.posteriors, axis=1)

        # Create binary labels indicating the component with the highest weight
        binary_labels = np.zeros_like(preds)
        for i in range(self.n_components):
            binary_labels[preds == i] = int(i == np.argmax(self.weights))
    
        return binary_label
        
        
        

### Part 2: Background Subtraction

![traffic](./videos/traffic.gif)

In this question, you are required to extract the background image from a given set of training frames, and use the extracted background to display foreground objects in the test frames by subtracting that background image and then thresholding it accordingly.

In this question, we are going to try different baselines to extract background from low resolution camera footage:

1. Frame Averaging:
    - Just take the average of every training frame, which gives us an approximate background image.
    
2. GMM Per Pixel:
    - We will maintain per pixel GMMs of 2 components, and then fit these GMMs considering every training from for its corresponding pixel.
    - And then use these GMMs to predict the pixel labels for every subsequent frame.
    - Most of the time, the Gaussian with the higher weight corresponds to the background.
    - We can implement this in a simpler way but with worse prediction results, you can extract a mean background image similar to the first baseline above.
    - To extract the Mean background image, we can assign values of the Means corresponding to the highest weighted Gaussian for each pixel.
    - This method is much simpler to implement but, this could give worse results.

#### Extracting Frames from videos

In [3]:
source_folder = 'videos'
video = 'traffic.gif'

source_path = f'./{source_folder}/{video}'

In [4]:
data_folder = 'frames'

frames_path = f"./{data_folder}/{video.rsplit('.', 1)[0]}"

In [5]:
%%capture

!mkdir -p {frames_path} > /dev/null ;

In [6]:
%%capture

!ffmpeg -i {source_path} {frames_path}/'frame_%04d.png' > /dev/null ;

#### Loading Frames

In [7]:
import glob

frames = []

for file_path in sorted(glob.glob(f'{frames_path}/*.png', recursive = False)):
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img = np.asarray(img, dtype=np.float64)
    img /= 255.0
    
    frames.append(img)
    
frames = np.asarray(frames, dtype=np.float64)

#### Splitting the data

In [8]:
from sklearn.model_selection import train_test_split

print(f'frame: {frames.shape}')

train_frames, test_frames = train_test_split(frames, train_size=0.6, shuffle=False) # Do Not Shuffle!

print(f'train_frames: {train_frames.shape}')
print(f'test_frames: {test_frames.shape}')

frame: (80, 120, 160, 3)
train_frames: (48, 120, 160, 3)
test_frames: (32, 120, 160, 3)


Note: You may use helper libraries like `imageio` for working with GIFs.

```python
import imageio

def make_gif(img_list, gif_path, fps=10):
    imageio.mimsave(gif_path, img_list, fps=fps) 
    return
```

#### Frame Averaging

Extract Background Image from the training data and display it.

In [9]:
# your code here
avg_background = np.mean(train_frames, axis=0)
cv2.imshow('Average background',avg_background)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### GMMs per pixel

Create Set of GMMs for every pixel and fit them considering every training frame

In [11]:
# your code here
start_time = time.time()
gmm_grid = [[None] * train_frames.shape[2] for _ in range(train_frames.shape[1])]
total_iterations = train_frames.shape[1]*train_frames.shape[2]
progress_bar = tqdm(total=total_iterations)
for i in range(train_frames.shape[1]):
    for j in range(train_frames.shape[2]):
        gmm = GMM(n_components=2)
        gmm.fit(train_frames[:,i,j,:])
        gmm_grid[i][j] = gmm
        progress_bar.update(1)
end_time = time.time()
elapsed_time = (end_time - start_time)/60
print("Elapsed time:", elapsed_time, "minutes")

# data = train_frames[:,1,1,:]
# data.shape
# gmm = GMM(n_components=2)
# gmm.fit(data)

 62%|██████████████████████████████████████████████████████████████████████████████▊                                               | 12000/19200 [10:06<06:04, 19.76it/s]/home/kumar/.local/lib/python3.10/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 19199/19200 [16:51<00:00, 28.63it/s]

Elapsed time: 16.86577180226644 minutes


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19200/19200 [17:10<00:00, 28.63it/s]

#### Extract Background Image from the trained model

In [12]:
# your code here
average_gmm = np.zeros((train_frames.shape[1],train_frames.shape[2],3))
for i in range(train_frames.shape[1]):
    for j in range(train_frames.shape[2]):
        gmm = gmm_grid[i][j]
        max_component_idx = np.argmax(gmm.weights)
        average_gmm[i,j,:] = gmm.means[max_component_idx]
cv2.imshow('GMM background',average_gmm)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Outputs

You can use the helper functions given below to display and save frames as videos, feel free to change them accordingly.

In [13]:
# helper functions

def display_frames(frames, fps=10.0):
    """
    Display the frames as a video.
    """
    eps = 0.0001
    
    wait_time = int(1000 // fps)
    
    for frame in frames:
        frame = frame.astype(np.float64)
        frame = (frame - frame.min()) * 255 / (frame.max() - frame.min() + eps)
        frame = frame.astype(np.uint8)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        cv2.imshow("video", frame)
        k = cv2.waitKey(wait_time)

        if k == ord('q'):
            print("Quitting...")
            break
    
    cv2.destroyAllWindows()


def save_frames(frames, fps=10.0, output_path='./results', file_name='temp'):
    """
    Save the frames as a video.
    """
    eps = 0.0001
    
    frame_rate = float(fps)
    frame_size = (int(frames[0].shape[1]), int(frames[0].shape[0]))
    wait_time = int(1000 // fps)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    save_path = os.path.join(output_path, f"{file_name.rsplit('.', 1)[0]}.mp4")
    
    vid_wrt = cv2.VideoWriter(save_path, fourcc, frame_rate, frame_size)

    for frame in frames:
        frame = frame.astype(np.float64)
        frame = (frame - frame.min()) * 255 / (frame.max() - frame.min() + eps)
        frame = frame.astype(np.uint8)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        cv2.imshow('frame',frame)
        if cv2.waitKey(wait_time) & 0xFF == ord('q'):
            break
        
        vid_wrt.write(frame)

        
    vid_wrt.release()
    cv2.destroyAllWindows()

#### Frame Averaging

In [17]:
# your output here
frames_bgremoved_avgbg = []
for frame in test_frames:
    frames_bgremoved_avgbg.append(frame-avg_background)
display_frames(frames_bgremoved_avgbg)
save_frames(frames_bgremoved_avgbg,file_name='average_background')

#### GMMs per pixel

In [18]:
# your output here
frames_bgremoved_gmmbg = []
for frame in test_frames:
    frames_bgremoved_gmmbg.append(frame-average_gmm)
display_frames(frames_bgremoved_gmmbg)
save_frames(frames_bgremoved_gmmbg,file_name='gmm_background')

In [24]:
#saving background images
avg_background_255 = (avg_background * 255)
cv2.imwrite('./results/average_background.png',avg_background_255)
average_gmm_255 = (average_gmm * 255)
cv2.imwrite('./results/gmm_background.png',average_gmm_255)

True